In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
from typing_extensions import Concatenate
# Function to read text from PDF files
import PyPDF2

def read_pdf(file_paths):
    raw_text = ''
    for file_path in file_paths:
        with open(file_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            raw_text+= str(file_path) + ' annual file report has started*****\n\n'
            for page in reader.pages:
                text = page.extract_text()
                raw_text += text
            raw_text+= str(file_path) + ' annual file report has ended***\n\n'
    return raw_text
#
# List of paths to PDF files
#pdf_files = ['Metro.pdf', 'Umar Farooq Khan.pdf']  # Add more file paths as needed

pdf_files = ["TEL_2023-annual-report.pdf", "VZ_2023-Annual-Report.pdf", "at&t_2023-complete-annual-report.pdf"]  # Add more file paths as needed

# Read text from PDF files
raw_text = read_pdf(pdf_files)

#print(raw_text)


In [ ]:

# Split the text using Character Text Split such that it should not increase token size
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
texts = text_splitter.split_text(raw_text)

# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings(openai_api_key='sk-proj-hh')
document_search = FAISS.from_texts(texts, embeddings)


chain = load_qa_chain(OpenAI(openai_api_key='sk-proj-hh'), chain_type="stuff")

In [ ]:


query = "Whats Diluted earnings per share of TEL company in 2023 and 2022?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The diluted earnings per share of TEL company in 2023 is $6.01 and in 2022 is $7.47.'

In [ ]:
query = "Which company of these have a good system devised for their long term employees, what are the benefits?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Verizon has a good system in place for their long term employees with benefits including pension benefits, retiree medical subsidy, and access to flexible and convenient health and welfare programs and workplace accommodations. They also prioritize employee development and training to support engagement and retention.'

In [ ]:
query = "What are the plans for these companies for the latin America?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The plans for the Latin America segment include network upgrades and expansion in 2024, as well as increasing operating income in 2023 and 2022. The segment also plans to provide fiber network internet services through a joint venture with Black Rock and continue to serve customers in Mexico through wireless data and voice communications services. They also hold interests in DIRECTV and SKY Mexico, both of which are leading pay-TV providers in Mexico.'